In [1]:
import keras
import numpy as np

from tensorflow.python.keras.callbacks import TensorBoard
from time import time
# Keras Module
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 64
np.random.seed(3)
batch_size = 10

In [3]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#img = load.img('C:/DataSet/train/0_non/flipme1.jpg')

In [4]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'C:/Dataset/train/',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=10,
        color_mode = 'grayscale',
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'C:/Dataset/test/',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),    
        batch_size=10,
        color_mode = 'grayscale',
        class_mode='categorical')

Found 2500 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(IMAGE_SIZE,IMAGE_SIZE, 1)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

print(model.summary())

tensorboard = TensorBoard(log_dir="./log/{}".format(time()))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        73792     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [6]:
#tb_hist = keras.callbacks.TensorBoard(log_dir='./log', histogram_freq=0, write_graph=True, write_images=True)
from keras import optimizers
LR= 1e-4

rms = keras.optimizers.RMSprop(lr=LR)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])


model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=test_generator,
        validation_steps=400 //batch_size,
        callbacks=[tensorboard])

#model.save_weights('./model/first.h5')
model.save('./model/model_softmax.h5')

print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=200)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

Epoch 1/50
200/200 [==============================] - 34s 168ms/step - loss: 0.6909 - acc: 0.5145 - val_loss: 0.6869 - val_acc: 0.5525
Epoch 2/50
200/200 [==============================] - 35s 177ms/step - loss: 0.6863 - acc: 0.5470 - val_loss: 0.6804 - val_acc: 0.5325
Epoch 3/50
200/200 [==============================] - 35s 176ms/step - loss: 0.6611 - acc: 0.6055 - val_loss: 0.6500 - val_acc: 0.6125
Epoch 4/50
200/200 [==============================] - 34s 172ms/step - loss: 0.6206 - acc: 0.6590 - val_loss: 0.6123 - val_acc: 0.6625
Epoch 5/50
200/200 [==============================] - 33s 166ms/step - loss: 0.5947 - acc: 0.6780 - val_loss: 0.5803 - val_acc: 0.6900
Epoch 6/50
200/200 [==============================] - 33s 166ms/step - loss: 0.5730 - acc: 0.6925 - val_loss: 0.5260 - val_acc: 0.7450
Epoch 7/50
200/200 [==============================] - 34s 168ms/step - loss: 0.5756 - acc: 0.6965 - val_loss: 0.5139 - val_acc: 0.7475
Epoch 8/50
200/200 [==============================] - 3

In [6]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=200)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 85.25%


In [7]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'drunken': 0, 'non': 1}
[[0.83149600 0.16850403]
 [0.91126537 0.08873464]
 [0.98319590 0.01680408]
 [0.87169474 0.12830526]
 [0.80510128 0.19489868]
 [0.99834633 0.00165368]
 [0.00652128 0.99347866]
 [0.06888712 0.93111283]
 [0.00697273 0.99302727]
 [0.07053022 0.92946976]
 [0.02431720 0.97568274]
 [0.00230009 0.99769992]
 [0.04192012 0.95807987]
 [0.82484132 0.17515872]
 [0.87579948 0.12420055]
 [0.25271770 0.74728233]
 [0.98827308 0.01172691]
 [0.93802422 0.06197583]
 [0.95558262 0.04441736]
 [0.97731119 0.02268881]
 [0.00822866 0.99177128]
 [0.01210202 0.98789793]
 [0.09070795 0.90929204]
 [0.01114457 0.98885542]
 [0.06210850 0.93789154]
 [0.11271532 0.88728470]
 [0.95333821 0.04666179]
 [0.15367676 0.84632325]
 [0.98961884 0.01038116]
 [0.03937734 0.96062273]
 [0.79506546 0.20493452]
 [0.09909296 0.90090698]
 [0.97684926 0.02315079]
 [0.04020862 0.95979136]
 [0.00626484 0.99373513]
 [0.95879209 0.04120794]
 [0.05002599 0.94997406]
 [0.99442261 0.00557735]
 [0.0431379

In [9]:
model.predict_classes(for i in test_generator)

SyntaxError: invalid syntax (<ipython-input-9-e99ed5e0c1fb>, line 1)

In [5]:
MODEL_NAME = "model_.h5"
WEIGHT_NAME = "first.h5"
MODEL_PATH = "./model/"
model = keras.models.load_model(MODEL_PATH + MODEL_NAME)
model.compile(loss='categorical_crossentropy', optimizer ='rmsprop', metrics =['accuracy'])

In [9]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.